In [1]:
import sys
import pandas as pd
import os

project_root = os.path.abspath(os.path.join(os.getcwd(), "../../"))

sys.path.append(project_root)
data_dir = os.path.join(project_root, 'data')

data_file_path = os.path.join(data_dir, 'RADCURE_processed_clinical.csv')
df = pd.read_csv(data_file_path, index_col=0)

df.drop(columns=["Study ID"], inplace=True)

In [2]:
from jarvais.analyzer import Analyzer
from rich import print

analyzer = Analyzer(
    data=df, 
    output_dir='./survival_outputs/analyzer',
    categorical_columns= [
      "Sex",
      "T Stage",
      "N Stage",
      "Stage",
      "Smoking Status",
      "Disease Site",
      "death",
      "HPV Combined",
      "Chemotherapy"
    ],
    continuous_columns = [
      "survival_time",
      "age at dx",
      "Dose"
    ],
    target_variable='death', 
    task='classification'
)

print(analyzer)

analyzer.run()

/home/joshua-siraj/Documents/CDI/jarvais/.pixi/envs/dev/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
14:41:01 [warning  ] Date columns not specified. Inferring from remaining columns. [jarvais] call=analyzer.__init__:76


Analyzer(
    AnalyzerSettings(
        output_dir=PosixPath('survival_outputs/analyzer'),
        categorical_columns=[
            'Sex',
            'T Stage',
            'N Stage',
            'Stage',
            'Smoking Status',
            'Disease Site',
            'death',
            'HPV Combined',
            'Chemotherapy'
        ],
        continuous_columns=['survival_time', 'age at dx', 'Dose'],
        date_columns=[],
        task='classification',
        target_variable='death',
        generate_report=True,
        settings_path=None,
        settings_schema_path=None,
        missingness=MissingnessModule(
            categorical_strategy={
                'Sex': 'unknown',
                'T Stage': 'unknown',
                'N Stage': 'unknown',
                'Stage': 'unknown',
                'Smoking Status': 'unknown',
                'Disease Site': 'unknown',
                'death': 'unknown',
                'HPV Combined': 'unknown',
                'Chemotherapy': 'unknown'
            },
            continuous_strategy={'survival_time': 'median', 'age at dx': 'median', 'Dose': 'median'},
            enabled=True
        ),
        outlier=OutlierModule(
            categorical_strategy={
                'Sex': 'frequency',
                'T Stage': 'frequency',
                'N Stage': 'frequency',
                'Stage': 'frequency',
                'Smoking Status': 'frequency',
                'Disease Site': 'frequency',
                'death': 'frequency',
                'HPV Combined': 'frequency',
                'Chemotherapy': 'frequency'
            },
            continuous_strategy={'survival_time': 'none', 'age at dx': 'none', 'Dose': 'none'},
            threshold=0.01,
            enabled=True
        ),
        encoding=OneHotEncodingModule(
            columns=[
                'Sex',
                'T Stage',
                'N Stage',
                'Stage',
                'Smoking Status',
                'Disease Site',
                'HPV Combined',
                'Chemotherapy'
            ],
            target_variable='death',
            prefix_sep='|',
            enabled=True
        ),
        visualization=VisualizationModule(
            plots=['corr', 'pairplot', 'umap', 'frequency_table', 'multiplot'],
            enabled=True
        )
    )
)

         [info     ] Performing missingness analysis... [jarvais] call=missingness.__call__:43
         [info     ] Performing outlier analysis... [jarvais] call=outlier.__call__:53
         [info     ] Plotting Correlation Matrix... [jarvais] call=visualization.__call__:115


+--------------------------+-------------------+-----------+-------------+
|                          |                   | Missing   | Overall     |
+==========================+===================+===========+=============+
| n                        |                   |           | 3346        |
+--------------------------+-------------------+-----------+-------------+
| survival_time, mean (SD) |                   | 0         | 4.1 (2.7)   |
+--------------------------+-------------------+-----------+-------------+
| age at dx, mean (SD)     |                   | 0         | 62.3 (11.6) |
+--------------------------+-------------------+-----------+-------------+
| Dose, mean (SD)          |                   | 0         | 66.7 (5.8)  |
+--------------------------+-------------------+-----------+-------------+
| Sex, n (%)               | Female            |           | 686 (20.5)  |
+--------------------------+-------------------+-----------+-------------+
|                        

         [info     ] Plotting Pairplot...           [jarvais] call=visualization.__call__:118
14:41:04 [info     ] Plotting UMAP...               [jarvais] call=visualization.__call__:124
14:41:13 [info     ] Plotting Frequency Table...    [jarvais] call=visualization.__call__:121
14:41:22 [info     ] Plotting Multiplot...          [jarvais] call=visualization.__call__:136
Font MPDFAA+Inter28ptBold is missing the following glyphs: '
' (\n)


In [3]:
from jarvais.trainer import TrainerSupervised

df = pd.read_csv('./survival_outputs/analyzer/updated_data.csv')
df.rename(columns={'survival_time': 'time', 'death':'event'}, inplace=True)

trainer = TrainerSupervised(task='survival', output_dir='./radcure_outputs/ED_trainer_explainer',)
trainer.run(df, ['event','time'])

Training MTLR...
  Best trial:
    Params: 
      C1: 0.01
      dropout: 0.3292835147016483
      dims: [64, 64]
Training DeepSurv...
  Best trial:
    Params: 
      l2_reg: 0.006451742136969566
      dropout: 0.360636124048913
      dims: [256, 256, 256]
Training CoxPH...
Training GradientBoosting...
Training RandomForest...
Training SVM...

Consolidated C-index Scores:
MTLR: 0.6482
DeepSurv: 0.6863
CoxPH: 0.7212
GradientBoosting: 0.7152
RandomForest: 0.7156
SVM: 0.7156


In [4]:
from jarvais.explainer import Explainer

exp = Explainer.from_trainer(trainer)
exp.run()

⚠️  **Possible Bias Detected in Disease Site** ⚠️
=== Subgroup Analysis for 'Disease Site' Using Cox Proportional Hazards Model ===

Model Statistics:
    AIC (Partial):               2393.09
    Log-Likelihood:              -1187.55
    Log-Likelihood Ratio p-value: 0.0003
    Concordance Index (C-index):   0.59
Model Coefficients:
    +--------------------------------+---------------+------------------+
    | Feature                        |   Coefficient |   Standard Error |
    +================================+===============+==================+
    | Disease Site_Other             |         0.424 |            4.571 |
    +--------------------------------+---------------+------------------+
    | Disease Site_esophagus         |         0.795 |            4.577 |
    +--------------------------------+---------------+------------------+
    | Disease Site_hypopharynx       |         0.746 |            4.556 |
    +--------------------------------+---------------+------------------+